In [1]:
import re
import pandas as pd
from openpyxl import load_workbook
pd.options.display.max_rows = None

In [5]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

# parse net name
def parse_net_name(string):
    # 定義正規表達式
    netNameRex = re.compile(r"^\s-?[A-Z]+[0-9]*_?[A-Z]+[0-9]*.*")
    netName = netNameRex.findall(string)
    return netName

# 找到總長度的欄位
def total(string):
    # 定義正規表達式
    totalRex = re.compile("TOTAL")
    total = totalRex.findall(string)
    return total
#找到位置
def parse_net(string):
#     locationRex = re.compile(r"([U|R|C|L|J]\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
#     locationRex = re.compile(r"([U|R|C|L|J][A-Z]+\d+\.[A-Z]*\d*|[U|R|C|L|J][A-Z]+\.[A-Z]*\d*|[U|R|C|L|J]\d+.[A-Z]*\d*|[U|R|C|L|J][A-Z]*\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    locationRex = re.compile(r"([U|R|C|L|J|T|P][A-Z]+\d+\.[A-Z]*\d*|[U|R|C|L|J|T|P][A-Z]+\.[A-Z]*\d*|[U|R|C|L|J|T|P|Q]\d+.[A-Z]*\d*|[U|R|C|L|J|T|P][A-Z]*\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    location = locationRex.findall(string)
    if location and location[0] == "VIA ":
        location[0] = location[0].replace(" ", "")
    return location

#找到單一段長度
def sub_length(string):
    lengthRex = re.compile(r"(\d+\.\d+)")
    length = lengthRex.findall(string)
    if length:
        length = length[-1]
    return length 

# 找到層數
def layer_num(string):
    layerRex = re.compile(r"BOTTOM|TOP|L\d+$")
    layer = layerRex.findall(string)
    return layer

#找到總長度
def total_length(string):
    ttlLength = re.findall(r"\d+.\d+", string)[0]
    return ttlLength

#存csv
def save_csv(datalist, filename):
    df = pd.DataFrame(datalist)
    df.to_csv(f'{filename}.csv', index = False)
    
# 存excel
def save_excel(write, df, sheetName):
#     df = pd.DataFrame(datalist)
    df.to_excel(write, sheet_name=f'{sheetName}', index=False)
    
#刪多餘的raw
def deleteNullVIAROW(df):
    for i in range(df.shape[0]):
        if df.loc[i, "location"] == "VIA" and df.loc[i, "gap"] == 0.0:
            df.drop(i, axis=0, inplace=True)
            
#這一份txt total的path 數量
def branchPathNum(df):
    #扣掉net name/start_end/total length
    maxPathNum = int((df.shape[1] - 3) / 2)
    return maxPathNum

In [6]:
#step1: 開啟檔案
filepath = r"0423.txt"
f = open_brd_file(filepath)
print(f"step1: {filepath} open.")

#step2: 解析檔案
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    #如果不是net name 也不是 total
    if not parse_net_name(i):
        if not parse_net(i):
            if not total(i):
                continue
            
    #如果是net name
    if parse_net_name(i):
        data = {} #清空
        netName = i.replace(" ", "")
        #建立netnamelist
        netNameList.append(netName)
        netPath = ""
#         data.update({"net_name" : netName})
#         SQS.append(data)
#         print(data)
    
    #如果是location
#     elif parse_net(i):
#         data = {} #清空
        
#         if sub_length(i):
#             location = parse_net(i)[0]
#             length = float(sub_length(i))
#             layer = layer_num(i)[0] if layer_num(i) else ""
#             data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
# #         print(data)
#             SQS.append(data)
    elif parse_net(i):
        data = {} #清空            
        if sub_length(i):
            location = parse_net(i)[0]
            
            length = float(sub_length(i))
            layer = layer_num(i)[0] if layer_num(i) else ""
            data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
            SQS.append(data)
        
    
    #如果是total那一行
    elif total(i):
        data = {} #清空
        ttlLength = float(total_length(i))
        data.update({"net_name" : netName, "total_length" : ttlLength})
#         print(data)
        summary.append(data)
        
print(f"step2: {filepath} parsed done.")


#存取df
dfsummary, dfSQS, dfSQSR = pd.DataFrame(summary), pd.DataFrame(SQS), pd.DataFrame(SQS)

step1: 0423.txt open.
step2: 0423.txt parsed done.


In [7]:
dfSQSR

,net_name,location,length,layer
0,-AOU_IFLG,U6321.9,0.000,
1,-AOU_IFLG,VIA,0.608,TOP
2,-AOU_IFLG,VIA,0.608,TOP
3,-AOU_IFLG,VIA,1090.911,L8
4,-AOU_IFLG,VIA,1400.519,L3
5,-AOU_IFLG,VIA,1400.519,TOP
6,-AOU_IFLG,R8601.1,1400.519,TOP
7,-AOU_IFLG,U23.B6,1539.579,TOP
8,-BATLOW,U23.A3,0.000,
9,-BATLOW,VIA,80.105,TOP


In [ ]:
ationRex = re.compile(r"(
                      [U|R|C|L|J|T|P][A-Z]+\d+\.[A-Z]*\d*
                      [U|R|C|L|J|T|P][A-Z]+\.[A-Z]*\d*
                      [U|R|C|L|J|T|P]\d+\.[A-Z]*\d*
                      [U|R|C|L|J|T|P][A-Z]*\d+\.[A-Z]*\d*
                      VIA\(T\)|VIA\s)")